In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import numpy as np
import ast

import iqplot


import bokeh.io
import bokeh.models
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
rg = np.random.default_rng()

def draw_bs_rep(data, func, rg):
    """Compute a bootstrap replicate from data."""
    bs_sample = rg.choice(data, size=len(data))
    return func(bs_sample)

def draw_bs_ci(data, func=np.mean, rg=rg, n_reps=2000):
    """Sample bootstrap multiple times and compute confidence interval"""
    bs_reps = np.array(
        [draw_bs_rep(data, func, rg) for _ in range(n_reps)]
    )
    conf_int = np.percentile(bs_reps,[2.5, 97.5])
    return(conf_int)

In [5]:
Data = pd.read_csv(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/DataSetAugust22.csv")

Data = Data[Data['Test Starvation'] == "Overnight no Water"]
Data = Data[Data["Training Starvation"] == "Not starved"]#.reset_index()
# Prep dataset from data
Melted = pd.melt(
    Data,
    id_vars=["Training", "ObjectsReinforced", "Reinforced_side", "Date", "Fly"],
    value_name="Durations",
    value_vars=[
        "Durations Left Corner",
        "Durations Right Corner",
        "Durations Top Corner",
        "Durations Left Front",
        "Durations Right Front",
        "Durations Top Front",
    ],
    var_name="Variable",
)

conditions = [
    (
        Melted["Reinforced_side"].str.contains("Right")
        & (Melted["Variable"].str.contains("Right"))
    ),
    (
        Melted["Reinforced_side"].str.contains("Left")
        & (Melted["Variable"].str.contains("Left"))
    ),
    (
        Melted["Reinforced_side"].str.contains("Right")
        & (Melted["Variable"].str.contains("Left"))
    ),
    (
        Melted["Reinforced_side"].str.contains("Left")
        & (Melted["Variable"].str.contains("Right"))
    ),
    (Melted["Variable"].str.contains("Top")),
]

values = [
    "Rewarded",
    "Rewarded",
    "Punished",
    "Punished",
    "Empty",
]
Melted["Condition"] = np.select(conditions, values)

conditions = [
    ((Melted["Variable"].str.contains("Corner"))),
    ((Melted["Variable"].str.contains("Front"))),
]
values = [
    "Corner",
    "Front",
]
Melted["Location"] = np.select(conditions, values)

In [8]:
Thresh = 80

for index, row in Melted.iterrows():
    Melted.loc[index, "Peeks"] = sum(
                1 for i in ast.literal_eval(row["Durations"]) if i > Thresh
            )

In [11]:
Stat_Data=Melted.groupby(['Training', 'Condition'])
Stat_Data

In [12]:

Groups = list(Stat_Data.groups.keys())

ConfInts = Stat_Data['Peeks'].apply(lambda x: draw_bs_ci(x))
Means = Stat_Data['Peeks'].mean()

In [15]:
Treshs = list(range(20, 300, 10))


In [85]:

CI_Data = pd.DataFrame


for t in Treshs:

        for index, row in Melted.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
            Melted.loc[index, "Peeks"] = sum(
                1 for i in ast.literal_eval(row["Durations"]) if i > t
            )
        Stat_Data=Melted.groupby(['Training', 'Condition'])
        Groups = list(Stat_Data.groups.keys())

        ConfInts = Stat_Data['Peeks'].apply(lambda x: draw_bs_ci(x))
        Means = Stat_Data['Peeks'].mean()

        C_Unst = ConfInts.unstack().reset_index()
        CMelt = pd.melt(
            C_Unst,
            id_vars=["Training",],
            value_name="CI",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        M_Unst = Means.unstack().reset_index()

        MMelt = pd.melt(
            M_Unst,
            id_vars=["Training",],
            value_name="Mean",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        Merged= pd.merge(CMelt, MMelt, on=["Training", "Condition"])

        Merged['Treshold'] = t

        yield(Merged)
        #CI_Data= pd.concat(CI_Data,Merged)







/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/3570711537.py:48: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  CI_Data= pd.concat(CI_Data,Merged)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "type"

In [91]:
CI_Data = pd.DataFrame


#for t in Treshs:
def TreshComp (Treshs):
    for t in Treshs:
        for index, row in Melted.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
            Melted.loc[index, "Peeks"] = sum(
                1 for i in ast.literal_eval(row["Durations"]) if i > t
            )
        Stat_Data=Melted.groupby(['Training', 'Condition'])
        Groups = list(Stat_Data.groups.keys())

        ConfInts = Stat_Data['Peeks'].apply(lambda x: draw_bs_ci(x))
        Means = Stat_Data['Peeks'].mean()

        C_Unst = ConfInts.unstack().reset_index()
        CMelt = pd.melt(
            C_Unst,
            id_vars=["Training",],
            value_name="CI",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        M_Unst = Means.unstack().reset_index()

        MMelt = pd.melt(
            M_Unst,
            id_vars=["Training",],
            value_name="Mean",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        Merged= pd.merge(CMelt, MMelt, on=["Training", "Condition"])

        Merged['Treshold'] = t

        yield(Merged)
        #CI_Data= pd.concat(CI_Data,Merged)

In [92]:
CI_Data = pd.concat(TreshComp(Treshs))


In [19]:
ConfInts["Ctrl", "Empty"][1]

7.103448275862069

In [29]:
Intervals = CI_Data['ConfInts']

In [40]:
Intervals.values

array([Training  Condition
       Ctrl      Empty          [27.41336206896552, 35.22456896551724]
                 Punished      [26.948275862068964, 35.43103448275862]
                 Rewarded     [24.724137931034484, 32.879741379310346]
       Trained   Empty                                [26.1385, 34.58]
                 Punished                             [29.32, 38.8805]
                 Rewarded                             [25.5795, 34.48]
       Name: Peeks, dtype: object                                     ,
       Training  Condition
       Ctrl      Empty        [22.257758620689653, 28.190517241379307]
                 Punished      [20.93103448275862, 27.138362068965513]
                 Rewarded       [18.93103448275862, 25.17241379310345]
       Trained   Empty                                [21.3795, 28.48]
                 Punished                  [22.48, 29.840999999999994]
                 Rewarded                             [19.28, 25.7805]
       Name: Peeks, dt

In [41]:
CI

[{'ConfInts': Training  Condition
  Ctrl      Empty          [27.41336206896552, 35.22456896551724]
            Punished      [26.948275862068964, 35.43103448275862]
            Rewarded     [24.724137931034484, 32.879741379310346]
  Trained   Empty                                [26.1385, 34.58]
            Punished                             [29.32, 38.8805]
            Rewarded                             [25.5795, 34.48]
  Name: Peeks, dtype: object,
  'Means': Training  Condition
  Ctrl      Empty        31.120690
            Punished     31.241379
            Rewarded     28.655172
  Trained   Empty        30.460000
            Punished     33.980000
            Rewarded     29.960000
  Name: Peeks, dtype: float64},
 {'ConfInts': Training  Condition
  Ctrl      Empty        [22.257758620689653, 28.190517241379307]
            Punished      [20.93103448275862, 27.138362068965513]
            Rewarded       [18.93103448275862, 25.17241379310345]
  Trained   Empty                  

In [44]:
TestExplo = CI_Data.explode(list('ConfIntsMeans'))

ValueError: column must be unique

In [45]:
TestExplo2 = TestExplo.explode('Means')

In [46]:
TestUnstack = CI_Data.unstack()

In [47]:
TestStack = CI_Data.stack()

In [53]:
TestUnstack2 = CI_Data.ConfInts.unstack

In [54]:
TestExplo3 = CI_Data.ConfInts.explode()

In [55]:
ConfInts

Training  Condition
Ctrl      Empty         [1.0517241379310345, 2.086206896551724]
          Punished     [0.8275862068965517, 1.5689655172413792]
          Rewarded      [0.930603448275862, 1.7413793103448276]
Trained   Empty                                    [1.32, 2.68]
          Punished                                 [0.98, 1.92]
          Rewarded                                 [0.88, 1.84]
Name: Peeks, dtype: object

In [56]:
ConfInts.Training

AttributeError: 'Series' object has no attribute 'Training'

In [65]:
#CMelt = C_Unst.melt()
C_Unst = ConfInts.unstack().reset_index()
CMelt = pd.melt(
    C_Unst,
    id_vars=["Training",],
    value_name="CI",
    value_vars=[
        "Empty",
        "Punished",
        "Rewarded"
    ],
    var_name="Condition",
)

In [67]:
Means

Training  Condition
Ctrl      Empty        1.551724
          Punished     1.172414
          Rewarded     1.327586
Trained   Empty        1.980000
          Punished     1.420000
          Rewarded     1.320000
Name: Peeks, dtype: float64

In [73]:
M_Unst = Means.unstack().reset_index()

MMelt = pd.melt(
    M_Unst,
    id_vars=["Training",],
    value_name="Mean",
    value_vars=[
        "Empty",
        "Punished",
        "Rewarded"
    ],
    var_name="Condition",
)

In [70]:
pd.concat([CMelt, MMelt])

,Training,Condition,CI
0,Ctrl,Empty,"[1.0517241379310345, 2.086206896551724]"
1,Trained,Empty,"[1.32, 2.68]"
2,Ctrl,Punished,"[0.8275862068965517, 1.5689655172413792]"
3,Trained,Punished,"[0.98, 1.92]"
4,Ctrl,Rewarded,"[0.930603448275862, 1.7413793103448276]"
5,Trained,Rewarded,"[0.88, 1.84]"
0,Ctrl,Empty,1.551724
1,Trained,Empty,1.98
2,Ctrl,Punished,1.172414
3,Trained,Punished,1.42


In [79]:
Merged= pd.merge(CMelt, MMelt, on=["Training", "Condition"])

Merged['Treshold'] = 20

In [78]:
pd.explode(Merged)

,Training,Condition,CI,Mean
0,Ctrl,Empty,1.051724,1.551724
0,Ctrl,Empty,2.086207,1.551724
1,Trained,Empty,1.32,1.980000
1,Trained,Empty,2.68,1.980000
2,Ctrl,Punished,0.827586,1.172414
2,Ctrl,Punished,1.568966,1.172414
3,Trained,Punished,0.98,1.420000
3,Trained,Punished,1.92,1.420000
4,Ctrl,Rewarded,0.930603,1.327586
4,Ctrl,Rewarded,1.741379,1.327586


In [93]:
CI_Data.CI[]

1               [26.1595, 34.3405]
1    [21.4995, 28.541499999999992]
1                    [15.18, 20.2]
1    [10.9195, 14.500499999999997]
1                    [8.26, 11.24]
1                     [6.44, 8.98]
1                     [5.42, 7.78]
1                      [4.12, 6.2]
1         [3.3, 5.240499999999997]
1                      [2.72, 4.6]
1      [2.4995, 4.320499999999997]
1                     [2.28, 4.04]
1                     [2.16, 3.86]
1                      [2.0, 3.76]
1                     [1.96, 3.66]
1                      [1.9, 3.54]
1        [1.78, 3.360499999999997]
1                     [1.66, 3.16]
1                     [1.66, 3.18]
1                      [1.6, 3.12]
1                     [1.52, 2.98]
1                      [1.5, 2.98]
1                     [1.42, 2.84]
1                     [1.36, 2.84]
1                     [1.34, 2.74]
1        [1.36, 2.780499999999997]
1       [1.32, 2.7404999999999973]
1                      [1.34, 2.7]
Name: CI, dtype: obj

In [95]:
ast.literal_eval(CI_Data["CI"][1])

ValueError: malformed node or string: 1               [26.1595, 34.3405]
1    [21.4995, 28.541499999999992]
1                    [15.18, 20.2]
1    [10.9195, 14.500499999999997]
1                    [8.26, 11.24]
1                     [6.44, 8.98]
1                     [5.42, 7.78]
1                      [4.12, 6.2]
1         [3.3, 5.240499999999997]
1                      [2.72, 4.6]
1      [2.4995, 4.320499999999997]
1                     [2.28, 4.04]
1                     [2.16, 3.86]
1                      [2.0, 3.76]
1                     [1.96, 3.66]
1                      [1.9, 3.54]
1        [1.78, 3.360499999999997]
1                     [1.66, 3.16]
1                     [1.66, 3.18]
1                      [1.6, 3.12]
1                     [1.52, 2.98]
1                      [1.5, 2.98]
1                     [1.42, 2.84]
1                     [1.36, 2.84]
1                     [1.34, 2.74]
1        [1.36, 2.780499999999997]
1       [1.32, 2.7404999999999973]
1                

In [97]:
CI_Data=CI_Data.reset_index()

In [98]:
ast.literal_eval(CI_Data["CI"][1])

ValueError: malformed node or string: array([26.1595, 34.3405])

In [101]:
CI_Data.CI[0][0]

27.396120689655174

In [104]:
LowerBounds = CI_Data.CI.str[0]
UpperBounds = CI_Data.CI.str[1]
Means = CI_Data.Mean

In [115]:
Emptys = CI_Data.loc[CI_Data.Condition == "Empty"]
EmptysC = Emptys.loc[Emptys.Training == "Ctrl"]

EmptysC['LowerBounds'] = EmptysC.CI.str[0]
EmptysC['UpperBounds'] = EmptysC.CI.str[1]

EmptysT = Emptys.loc[Emptys.Training == "Trained"]

EmptysT['LowerBounds'] = EmptysT.CI.str[0]
EmptysT['UpperBounds'] = EmptysT.CI.str[1]



/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/3752814992.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EmptysC['LowerBounds'] = EmptysC.CI.str[0]
/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/3752814992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EmptysC['UpperBounds'] = EmptysC.CI.str[1]
/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/3752814992.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [118]:
Rewardeds = CI_Data.loc[CI_Data.Condition == "Rewarded"]
RewardedsC = Rewardeds.loc[Rewardeds.Training == "Ctrl"]

RewardedsC['LowerBounds'] = RewardedsC.CI.str[0]
RewardedsC['UpperBounds'] = RewardedsC.CI.str[1]

RewardedsT = Rewardeds.loc[Rewardeds.Training == "Trained"]

RewardedsT['LowerBounds'] = RewardedsT.CI.str[0]
RewardedsT['UpperBounds'] = RewardedsT.CI.str[1]


/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/2690872476.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RewardedsC['LowerBounds'] = RewardedsC.CI.str[0]
/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/2690872476.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RewardedsC['UpperBounds'] = RewardedsC.CI.str[1]
/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/2690872476.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [129]:
Punishes = CI_Data.loc[CI_Data.Condition == "Punished"]
PunishesC = Punishes.loc[Punishes.Training == "Ctrl"]

PunishesC['LowerBounds'] = PunishesC.CI.str[0]
PunishesC['UpperBounds'] = PunishesC.CI.str[1]

PunishesT = Punishes.loc[Punishes.Training == "Trained"]

PunishesT['LowerBounds'] = PunishesT.CI.str[0]
PunishesT['UpperBounds'] = PunishesT.CI.str[1]


/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/1772253791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PunishesC['LowerBounds'] = PunishesC.CI.str[0]
/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/1772253791.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PunishesC['UpperBounds'] = PunishesC.CI.str[1]
/var/folders/3v/_y6xq12x6l96bm2kp__gv6z40000gn/T/ipykernel_5081/1772253791.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [130]:
x = 'Peeking Threshold'
y = 'Count'

# Make the plot
s1 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Empty chamber peekings',
)

s1.circle(
    source=EmptysC,
    x='Treshold',
    y='Mean',
)

s1.segment(
    source=EmptysC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s1.circle(
    source=EmptysT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s1.segment(
    source=EmptysT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)


###
s2 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Rewarded chamber peekings',

)

s2.circle(
    source=RewardedsC,
    x='Treshold',
    y='Mean',
)

s2.segment(
    source=RewardedsC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s2.circle(
    source=RewardedsT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s2.segment(
    source=RewardedsT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)

###
s3 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Punished chamber peekings',

)

s3.circle(
    source=PunishesC,
    x='Treshold',
    y='Mean',
)

s3.segment(
    source=PunishesC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s3.circle(
    source=PunishesT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s3.segment(
    source=PunishesT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)
#p = bokeh.io.hplot(s1,s2)


bokeh.io.show(bokeh.layouts.column(s1,s2,s3))
